In [105]:
import json
import boto3
import pandas as pd
#import mysql.connector
from collections import OrderedDict
from datetime import datetime, timedelta

In [81]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    keys = []
    resp = s3.list_objects_v2(Bucket=bucket)
    for obj in resp['Contents']:
        keys.append(obj['Key'])
    return keys

In [82]:
s3 = boto3.client('s3')

In [83]:
bucket = 'scc-enrollment-data'

In [93]:
files = get_s3_keys(bucket)

In [94]:
idx = -1
files[idx]

'SCC_Enrollment_MA_2020_02.csv'

In [95]:
obj = s3.get_object(Bucket= bucket, Key= files[idx])

In [87]:
latest_df = pd.read_csv(obj['Body']) # 'Body' is a key word

#### Import MA Enrollment data

In [103]:
_dict = {}
for file in reversed(files):
    k = len(_dict)
    print(file)
    obj = s3.get_object(Bucket= bucket, Key= file)
    df = pd.read_csv(obj['Body'])
    df['Contract ID'] = df['Contract ID'].astype(str)
    df['SSA Code'] = df['SSA Code'].astype(str)
    df['FIPS Code'] = df['FIPS Code'].fillna(99999).astype(int).round(0).astype(str)
    df['FIPS Code'] = df['FIPS Code'].apply(lambda x: x.zfill(5))
    df['unique_id'] = df['Contract ID'] + df['SSA Code']
    df.drop(['Enrolled'], axis=1)
    for _ , row in df.iterrows():
        if row['unique_id'] not in _dict:
            _dict[row['unique_id']] = {'County': row['County'], 'State': row['State'], 'Contract ID': row['Contract ID'],
                              'Organization': row['Organization Name'], 'Type': row['Organization Type'], 'Plan Type': row['Plan Type'],
                              'SSA Code': row['SSA Code'], 'FIPS Code': row['FIPS Code']}
        else:
            pass
    m = len(_dict)
    print("Just added ", m - k, " keys")
_dict

SCC_Enrollment_MA_2020_02.csv
Just added  438878  keys
SCC_Enrollment_MA_2020_01.csv
Just added  5841  keys
SCC_Enrollment_MA_2019_12.csv
Just added  6612  keys
SCC_Enrollment_MA_2019_11.csv
Just added  4448  keys
SCC_Enrollment_MA_2019_10.csv
Just added  3709  keys
SCC_Enrollment_MA_2019_09.csv
Just added  2970  keys
SCC_Enrollment_MA_2019_08.csv
Just added  3382  keys
SCC_Enrollment_MA_2019_07.csv
Just added  2679  keys
SCC_Enrollment_MA_2019_06.csv
Just added  2890  keys
SCC_Enrollment_MA_2019_05.csv
Just added  2849  keys
SCC_Enrollment_MA_2019_04.csv
Just added  2582  keys
SCC_Enrollment_MA_2019_03.csv
Just added  2726  keys
SCC_Enrollment_MA_2019_02.csv
Just added  2987  keys
SCC_Enrollment_MA_2019_01.csv
Just added  2884  keys
SCC_Enrollment_MA_2018_12.csv
Just added  13329  keys
SCC_Enrollment_MA_2018_11.csv
Just added  3055  keys
SCC_Enrollment_MA_2018_10.csv
Just added  2626  keys
SCC_Enrollment_MA_2018_09.csv
Just added  2572  keys
SCC_Enrollment_MA_2018_08.csv
Just added  2

{'900911000': {'County': 'Autauga',
  'State': 'AL',
  'Contract ID': '90091',
  'Organization': 'UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT',
  'Type': 'HCPP - 1833 Cost',
  'Plan Type': 'HCPP - 1833 Cost',
  'SSA Code': '1000',
  'FIPS Code': '01001'},
 'H01041000': {'County': 'Autauga',
  'State': 'AL',
  'Contract ID': 'H0104',
  'Organization': 'BLUE CROSS AND BLUE SHIELD OF ALABAMA',
  'Type': 'Local CCP',
  'Plan Type': 'Local PPO',
  'SSA Code': '1000',
  'FIPS Code': '01001'},
 'H01071000': {'County': 'Autauga',
  'State': 'AL',
  'Contract ID': 'H0107',
  'Organization': 'HEALTH CARE SERVICE CORPORATION',
  'Type': 'Local CCP',
  'Plan Type': 'Local PPO',
  'SSA Code': '1000',
  'FIPS Code': '01001'},
 'H01541000': {'County': 'Autauga',
  'State': 'AL',
  'Contract ID': 'H0154',
  'Organization': 'VIVA HEALTH, INC.',
  'Type': 'Local CCP',
  'Plan Type': 'HMO/HMOPOS',
  'SSA Code': '1000',
  'FIPS Code': '01001'},
 'H04321000': {'County': 'Autauga',
  'State': 'AL',
  '

##### Save the dict as a json object

In [ ]:
with open('unique_contracts.json', 'w') as file:
    file.write(json)

##### Import the json as a dict

In [120]:
with open('contract_id.json', 'r') as file:
    data = json.loads(file.read())

In [ ]:
df = pd.DataFrame.from_csv('SCC_Enrollment_MA_2020_02.csv', index_col=None)

###### Feature Engineering our columns

In [97]:
df['Contract ID'] = df['Contract ID'].astype(str)
df['SSA Code'] = df['SSA Code'].astype(str)
df['FIPS Code'] = df['FIPS Code'].fillna(99999).astype(int).round(0).astype(str)
df['FIPS Code'] = df['FIPS Code'].apply(lambda x: x.zfill(5))
df['unique_id'] = df['Contract ID'] + df['SSA Code']
df.drop(['Enrolled'], axis=1)

,County,State,Contract ID,Organization Name,Organization Type,Plan Type,SSA Code,FIPS Code,unique_id
0,Autauga,AL,90091,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,HCPP - 1833 Cost,HCPP - 1833 Cost,1000,01001,900911000
1,Autauga,AL,H0104,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Local CCP,Local PPO,1000,01001,H01041000
2,Autauga,AL,H0107,HEALTH CARE SERVICE CORPORATION,Local CCP,Local PPO,1000,01001,H01071000
3,Autauga,AL,H0154,"VIVA HEALTH, INC.",Local CCP,HMO/HMOPOS,1000,01001,H01541000
4,Autauga,AL,H0432,"UNITEDHEALTHCARE OF ALABAMA, INC.",Local CCP,HMO/HMOPOS,1000,01001,H04321000
5,Autauga,AL,H0504,CALIFORNIA PHYSICIANS' SERVICE,Local CCP,HMO/HMOPOS,1000,01001,H05041000
6,Autauga,AL,H0523,AETNA HEALTH OF CALIFORNIA INC.,Local CCP,HMO/HMOPOS,1000,01001,H05231000
7,Autauga,AL,H0524,"KAISER FOUNDATION HP, INC.",Local CCP,HMO/HMOPOS,1000,01001,H05241000
8,Autauga,AL,H0628,AETNA HEALTH OF OHIO INC.,Local CCP,HMO/HMOPOS,1000,01001,H06281000
9,Autauga,AL,H0710,"SIERRA HEALTH AND LIFE INSURANCE COMPANY, INC.",Local CCP,Local PPO,1000,01001,H07101000


###### If the unique id does not exist in our dictionary, add it

In [78]:
_dict = {}
for index, row in df.iterrows():
    if row['unique_id'] not in _dict:
        _dict[row['unique_id']] = {'County': row['County'], 'State': row['State'], 'Contract ID': row['Contract ID'],
                              'Organization': row['Organization Name'], 'Type': row['Organization Type'], 'Plan Type': row['Plan Type'],
                              'SSA Code': row['SSA Code'], 'FIPS Code': row['FIPS Code']}
    else:
        pass

###### When finished, export to dataframe

In [ ]:
with open('unique_contracts.json', 'w') as file:
    file.write(json)